In [1]:
   import torch
   import torch.nn as nn
   import torch.optim as optim
   import torchvision.transforms as transforms
   import torchvision.datasets as datasets
   from torch.utils.data import DataLoader
   import pandas as pd

In [21]:
train_dir = f"C:\\Users\\PC\\Desktop\\Projects\\Dates-classification\\Dates Classification\\data\\train"
labels_file = f"C:\\Users\\PC\\Desktop\\Projects\\Dates-classification\\Dates Classification\\data\\train_labels.csv"
test_dir = f"C:\\Users\\PC\\Desktop\\Projects\\Dates-classification\\Dates Classification\\data\\test"

In [13]:
class SimpleNet(nn.Module):
       def __init__(self):
           super(SimpleNet, self).__init__()
           self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
           self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
           self.fc1 = nn.Linear(32 * 8 * 8, 128)  
           self.fc2 = nn.Linear(128, 7)  

       def forward(self, x):
           x = F.relu(self.conv1(x))
           x = F.max_pool2d(x, 2)
           x = F.relu(self.conv2(x))
           x = F.max_pool2d(x, 2)
           x = x.view(x.size(0), -1)
           x = F.relu(self.fc1(x))
           x = self.fc2(x)
           return x

In [14]:
net = SimpleNet()

In [15]:
labels_df = pd.read_csv(labels_file)

class_names = labels_df['label'].unique()  
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}  # Create mapping

print(class_to_idx)

{'Ajwa': 0, 'Medjool': 1, 'Meneifi': 2, 'Nabtat Ali': 3, 'Shaishe': 4, 'Sokari': 5, 'Sugaey': 6}


In [16]:
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, labels_file, transform=None):
        self.image_dir = image_dir
        self.labels_df = pd.read_csv(labels_file)
        self.transform = transform
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.labels_df['label'].unique())}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.labels_df.iloc[idx, 0])  
        image = Image.open(img_name).convert("RGB")  
        label = self.labels_df.iloc[idx, 1]  
        label_idx = self.class_to_idx[label]  
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label_idx) 

class CustomTestImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = os.listdir(image_dir)  
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_filenames[idx])  
        image = Image.open(img_name).convert("RGB")  

        if self.transform:
            image = self.transform(image)

        return image, self.image_filenames[idx]  #

In [17]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_dataset = CustomImageDataset(image_dir=train_dir, labels_file=labels_file, transform=transform)

test_dataset = CustomTestImageDataset(image_dir=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    net.train()  

    for images, labels in train_loader:
        optimizer.zero_grad()  
        outputs = net(images)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [ ]:
net.eval()

predictions = []
with torch.no_grad():  
       for images, filenames in test_loader:
           outputs = net(images)  s
           _, predicted = torch.max(outputs, 1)  
           # Collect predictions
           for i in range(len(filenames)):
               predictions.append((filenames[i], predicted[i].item()))  